In [8]:
import pymongo
from pymongo import MongoClient
from pymongo import GEO2D, GEOSPHERE
from pymongo import GEO2D
from datetime import datetime
from pprint import pprint
from bson.objectid import ObjectId
import re
import string
from geojson import Point, Polygon, MultiPolygon

from pymongo.mongo_client import MongoClient
import datetime
import json
#if you don't have geo json installed, use pip install geojson
#if you want to start mongodb use:  sudo service mongod start
#Monday is 0 -> week day

In [2]:
%%bash
service mongod status

mongod start/running, process 15470


In [13]:
client = MongoClient('mongodb://localhost:27017')
db = client.MyDB
db.collection_names()
client.database_names()

['green_may']

In [22]:
#Show that current collection doesn't have neigh info
collection = db['green_may']
cc=collection.find(limit=5)
for c in cc:
    pprint(c)

{'_id': ObjectId('5a4e4c84f06accbe42532f24'), 'lpep_pickup_datetime': datetime.datetime(2014, 5, 1, 0, 0), 'Lpep_dropoff_datetime': datetime.datetime(2014, 5, 1, 10, 50, 16), 'Passenger_count': 1, 'Trip_distance': 5.65, 'Fare_amount': 26.5, 'Total_amount': 30.0, 'Payment_type': 1, 'dropoff_Point': {'type': 'Point', 'coordinates': [0.0, 0.0]}, 'pickup_Point': {'type': 'Point', 'coordinates': [0.0, 0.0]}, 'pickup_weekday': 3, 'dropoff_weekday': 3}


In [14]:
# If the neigh collection doesn't exist, load the neighborhoods file into a Json Object, then add to a collection
# nyc_neighborhoods.json should be downloaded in the current working directory
neighborhoods = json.load(open('nyc_neighborhoods.json'))
ncollection = db['neighborhoods']
for f in neighborhoods['features']:
    bcode = int(f["properties"]['boroughCode'])
    n = {'boroughCode': bcode,\
         'borough': f["properties"]['borough'],\
         'neighborhood': f["properties"]['neighborhood'],\
         'coordinates': f["geometry"]['coordinates'][0]}
    ncollection.insert_one(n)
ncollection.find_one()

{'_id': ObjectId('5a52181febabc7016fa51010'),
 'borough': 'Bronx',
 'boroughCode': 2,
 'coordinates': [[-73.84859700000018, 40.871670000000115],
  [-73.84582253683678, 40.870239076236174],
  [-73.85455918463374, 40.85995383576425],
  [-73.85466543306826, 40.859585694988056],
  [-73.85638870335896, 40.85759363530448],
  [-73.86888180915341, 40.857223150158326],
  [-73.86831755272824, 40.85786206225831],
  [-73.86955371467232, 40.85778409560018],
  [-73.87102485762065, 40.857309948816905],
  [-73.87048054998716, 40.865413584098484],
  [-73.87055489856489, 40.86970279858986],
  [-73.86721594442561, 40.86968966363671],
  [-73.85745, 40.86953300000018],
  [-73.85555000000011, 40.871813000000145],
  [-73.85359796757658, 40.8732883686742],
  [-73.84859700000018, 40.871670000000115]],
 'neighborhood': 'Allerton'}

In [17]:
# Adds the dropoff neigh info (boroughcode and neigh name) to the taxi ride documents
countIgnored = 0 #count of taxi rides with information already added
countUpdated = 0 # count of taxi rides that have been updated
cursor = ncollection.find().batch_size(10)
bulk = collection.initialize_unordered_bulk_op()
for poly in cursor:
    bCode = poly["boroughCode"]
    nName = poly["neighborhood"]
    pprint(poly["borough"])
    pprint(nName)
    pprint(countUpdated)
    taxis = collection.find({"dropoff_Point": {"$geoWithin": {"$polygon": poly['coordinates']}}})
    for t in taxis:
        if 'dropoff_b_code' in t:
            countIgnored += 1
            continue
        countUpdated += 1
        bulk.find({ '_id': t['_id']}).update({'$set':{'dropoff_b_code' : bCode, 'dropoff_n_Name' : nName}})
        if countUpdated % 1000 == 0:
            pprint(countUpdated)
            bulk.execute()
            bulk = collection.initialize_unordered_bulk_op()
if (countUpdated % 1000 != 0):
        bulk.execute()
pprint('Updated')
pprint(countUpdated)
pprint('Ignored')
pprint(countIgnored)


'Bronx'
'Allerton'
0
1000
'Queens'
'Alley Pond Park'
1995
2000
'Staten Island'
'Arden Heights'
2008
'Staten Island'
'Arlington'
2014
'Staten Island'
'Arrochar'
2014
'Queens'
'Arverne'
2023
'Queens'
'Astoria'
2051
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
'Brooklyn'
'Bath Beach'
51081
'Manhattan'
'Battery Park City'
51395
52000
53000
'Brooklyn'
'Bay Ridge'
53427
54000
55000
'Queens'
'Bay Terrace'
55324
'Staten Island'
'Bay Terrace, Staten Island'
55610
'Bronx'
'Baychester'
55610
56000
'Queens'
'Bayside'
56028
57000
'Queens'
'Bayswater'
57436
'Queens'
'Bayswater'
57473
'Brooklyn'
'Bedford-Stuyvesant'
57473
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000

591000
592000
593000
594000
595000
596000
597000
598000
599000
600000
601000
602000
603000
604000
605000
606000
607000
608000
609000
610000
611000
612000
613000
614000
615000
616000
617000
618000
619000
620000
621000
622000
623000
624000
625000
626000
627000
628000
629000
630000
631000
632000
633000
634000
635000
636000
637000
638000
639000
640000
641000
642000
643000
644000
645000
646000
647000
648000
649000
650000
651000
652000
653000
654000
655000
656000
657000
658000
659000
660000
661000
662000
663000
664000
665000
666000
667000
668000
669000
670000
671000
672000
673000
674000
675000
676000
677000
678000
679000
680000
681000
682000
683000
684000
685000
686000
687000
688000
689000
'Bronx'
'Hart Island'
689134
'Manhattan'
"Hell's Kitchen"
689134
690000
691000
692000
693000
694000
695000
696000
697000
698000
699000
700000
701000
702000
703000
'Bronx'
'Highbridge'
703961
704000
705000
706000
707000
708000
709000
'Staten Island'
'Hoffman Island'
709009
'Queens'
'Hollis'
709009
'Queens'


'Bronx'
'South Brother Island'
1093784
'Queens'
'South Ozone Park'
1093784
1094000
1095000
1096000
'Brooklyn'
'South Slope'
1096466
1097000
1098000
1099000
1100000
1101000
1102000
'Queens'
'Springfield Gardens'
1102714
1103000
'Bronx'
'Spuyten Duyvil'
1103540
1104000
'Queens'
'St. Albans'
1104089
1105000
'Staten Island'
'St. George'
1105161
'Staten Island'
'Stapleton'
1105191
'Manhattan'
'Stuyvesant Town'
1105207
1106000
'Queens'
'Sunnyside'
1106523
1107000
1108000
1109000
1110000
1111000
1112000
1113000
1114000
1115000
1116000
1117000
1118000
1119000
1120000
1121000
'Brooklyn'
'Sunset Park'
1121408
1122000
1123000
1124000
1125000
1126000
1127000
1128000
1129000
'Manhattan'
'Theater District'
1129098
1130000
1131000
1132000
1133000
1134000
1135000
'Bronx'
'Throgs Neck'
1135979
1136000
1137000
'Staten Island'
'Todt Hill'
1137396
'Staten Island'
'Tompkinsville'
1137405
'Staten Island'
'Tottenville'
1137415
'Bronx'
'Tremont'
1137416
1138000
1139000
1140000
'Manhattan'
'Tribeca'
1140332
11

In [15]:
# Adds the pickup neigh info (boroughcode and neigh name) to the taxi ride documents
countIgnored = 0 #count of taxi rides with information already added
countUpdated = 0 # count of taxi rides that have been updated
cursor = ncollection.find().batch_size(10)
bulk = collection.initialize_unordered_bulk_op()
for poly in cursor:
    bCode = poly["boroughCode"]
    nName = poly["neighborhood"]
    pprint(poly["borough"])
    pprint(nName)
    pprint(countUpdated)
    taxis = collection.find({"pickup_Point": {"$geoWithin": {"$polygon": poly['coordinates']}}})
    for t in taxis:
        if 'pickup_b_code' in t:
            countIgnored += 1
            continue
        countUpdated += 1
        bulk.find({ '_id': t['_id']}).update({'$set':{'pickup_b_code' : bCode, 'pickup_n_Name' : nName}})
        if countUpdated % 1000 == 0:
            pprint(countUpdated)
            bulk.execute()
            bulk = collection.initialize_unordered_bulk_op()
if (countUpdated % 1000 != 0):
        bulk.execute()
pprint('Updated')
pprint(countUpdated)
pprint('Ignored')
pprint(countIgnored)


2
'Allerton'
0
1000
4
'Alley Pond Park'
1218
5
'Arden Heights'
1226
5
'Arlington'
1228
5
'Arrochar'
1228
4
'Arverne'
1231
4
'Astoria'
1239
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
3
'Bath Beach'
78414
1
'Battery Park City'
78478
3
'Bay Ridge'
78484
79000
4
'Bay Terrace'
79165
5
'Bay Terrace, Staten Island'
79192
2
'Baychester'
79192
4
'Bayside'
79427
4
'Bayswater'
79757
4
'Bayswater'
79764
3
'Bedford-Stuyvesant'
79764
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106

751000
752000
753000
754000
755000
756000
757000
758000
759000
760000
761000
762000
763000
764000
765000
766000
767000
768000
769000
770000
771000
772000
773000
774000
775000
776000
777000
778000
779000
780000
781000
782000
783000
784000
785000
786000
787000
788000
789000
790000
791000
792000
793000
794000
795000
796000
797000
798000
799000
800000
801000
802000
803000
804000
805000
806000
807000
808000
809000
810000
811000
812000
813000
814000
815000
816000
817000
818000
819000
820000
2
'Hart Island'
820444
1
"Hell's Kitchen"
820444
2
'Highbridge'
820483
821000
822000
823000
5
'Hoffman Island'
823663
4
'Hollis'
823663
4
'Hollis Hills'
823794
4
'Holliswood'
823801
4
'Howard Beach'
823831
4
'Howard Beach'
823831
5
'Howland Hook'
823893
5
'Huguenot'
823895
2
'Hunts Point'
823895
824000
1
'Inwood'
824356
825000
826000
827000
828000
829000
830000
831000
832000
833000
834000
835000
836000
837000
4
'Jackson Heights'
837083
838000
839000
840000
841000
842000
843000
844000
845000
846000
847000


1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000
1289000
1290000
1291000
1292000
1293000
1294000
1295000
1296000
1297000
1298000
1299000
1300000
1301000
1302000
1303000
1304000
1305000
1306000
1307000
1308000
1309000
1310000
1311000
1312000
1313000
1314000
1315000
1316000
1317000
1318000
1319000
1320000
1321000
1322000
1323000
1324000
1325000
1326000
1327000
1328000
1329000
1330000
1331000
1332000
1333000
1334000
1335000
1336000
1337000
1338000
1339000
1340000
1341000
1342000
1343000
1344000
1345000
1346000
1347000
1348000
1349000
1350000
1351000
1352000
1353000
1354000
1355000
1356000
1357000
1358000
1359000
1360000
1361000
1362000
1363000
1364000
1365000
1366000
1367000
1368000
1369000
1370000
1371000
1372000
1373000
1374000
1375000
1376000
1377000
1378000
1379000
1380000
5
'Willowbrook'
1380883
3
'Windsor Terrace'
1380883
1381000
1382000
4
'Woodhaven'
1382098
2
'Woodlawn'
1382573
5
'Woodro